In [ ]:
'''backtest
start: 2018-11-01 00:00:00
end: 2019-07-10 00:00:00
period: 1h
basePeriod: 15m
exchanges: [{"eid":"Binance","currency":"BTC_USDT","balance":10000,"stocks":1}]
'''

class midClass():
    def __init__(self,thisExchange):
        '''
        初始化数据 交易所信息， 首次获取价格，首次获取account信息
        
        '''
        self.initTimestamp = time.time()
        self.exchange = thisExchange
        self.name = self.exchange.GetName()
        self.coin = self.exchange.GetCurrency()
        Log('self.name',self.name )
        Log('self.coin',self.coin )
    def getAccount(self):
        '''
        获取账户信息
        '''
        try:
            #账户信息初始化
            self.balance = '---'
            self.amount = '---'
            self.frozenBalance = '---'
            self.frozenStocks = '---'

            #交易所信息
            self.account = self.exchange.GetAccount()
            Log(self.account )
            #账户信息赋值
            self.balance = self.account['Balance']
            self.frozenBalance = self.account['FrozenBalance']
            self.amount = self.account['Stocks']
            self.frozenStocks = self.account['FrozenStocks']
            return True
        except:
            return False
        
        pass
    def getTicker(self):
        '''
        获取市价信息
        '''
        try:
            ticker = exchange.GetTicker()
            # 异常行情不交易，如扎针
            #if random.random()*100 < 50:
            #    if ticker["High"] >= self.tickerHigh * 1.5:
            #    return 'exception ticker'
            self.tickerHigh = ticker["High"];
            self.tickerLow = ticker["Low"];
            self.tickerSell = ticker["Sell"];
            self.tickerBuy = ticker["Buy"];
            self.tickerLast = ticker["Last"];
            self.tickerVolume = ticker["Volume"];
            return True
        except:
            return False
        pass
    def getDepth(self):
        '''
        获取深度
        '''
        try:
            depth = exchange.GetDepth()
#             price = depth["Asks"][1]["Price"]
            self.depthAsk = depth["Asks"]
            self.depthBids = depth["Bids"]
            return True
        except:
            return False
        pass
    def getOhlcData(self,period = PERIOD_M15):
        '''
        获取K线信息
        '''
        try:
            records = exchange.GetRecords(period)
            return records
        except:
            return False
        pass
    def createOrder(self,orderType,orderPrice,orderNum):
        '''
        下单
        '''
        try:
            if orderType == 'buy':
                id = self.exchange.Buy(orderPrice, orderNum)
            elif orderType == 'sell':
                id = self.exchange.Sell(orderPrice, orderNum)
                
            return True
        except:
            return False         
        pass
    def cancelOrder(self,orderId):
        '''
        取消挂单
        '''
        
        return self.exchange.CancelOrder(orderId)
        pass
    def refleshData(self, period = PERIOD_M15):
        '''
        刷新信息
        '''
        if not self.getAccount():
            return 'excep getAccount'
        if not self.getTicker():
            return 'excep getAccount'
        if not self.getDepth():
            return 'excep getAccount'    
        if not self.getOhlcData(period):   
            return 'excep getAccount'    
        
        
class averageBalance():
    def __init__(self,midClass,period = PERIOD_M15):
        self.jys = midClass
        self.lastTime = self.jys.initTimestamp
        self.lastTickerLast = self.jys.tickerLast
        self.buyCount = 0
        self.sellCount = 0
    
    def makeNeedAccountInfo(self,period = PERIOD_M15):
        err = self.jys.refleshData(period)
        if(err) :
            Log(err)
            return 'error'
        self.balance = self.jys.balance
        self.amount = self.jys.amount
        Log('self.balance',self.balance,'self.amount',self.amount)
        self.halfBalance = (self.balance + self.amount) / 2
        self.tickerLast = self.jys.tickerLast
        self.tickerHigh = self.jys.tickerHigh
        self.tickerLow = self.jys.tickerLow
        self.needBuyAmount = (self.halfBalance - self.tickerLast * self.amount) / self.tickerLast
        self.needSellAmount = (self.tickerLast * self.amount - self.halfBalance) / self.tickerLast
        
        pass
    def run(self, type, value):
        if type == 'time':
            if time.time() - self.lastTime > value:
                self.doAverge()
                self.lastTime = time.time()
        if type == 'price':
            Log('(abs(self.lastTickerLast - self.tickerLast) / self.tickerLast)',(abs(self.lastTickerLast - self.tickerLast) / self.tickerLast))
            if (abs(self.lastTickerLast - self.tickerLast) / self.tickerLast) > value:
                self.doAverge()
                self.lastTickerLast = self.tickerLast
        
    def doAverge(self):
#         if condition == 'time':
#             if()
        if self.needBuyAmount > 0.001 :
            self.buyCount += 1
            self.jys.createOrder('buy', self.tickerLow, self.needBuyAmount)
        elif self.needSellAmount > 0.001 :
            self.sellCount += 1
            self.jys.createOrder('sell', self.tickerHigh, self.needSellAmount)
            
        Log('sellCount:',self.sellCount,'buyCount:',self.buyCount)
        pass
def main():
    theMidClass = midClass(exchange)
    Log(theMidClass.refleshData());
    theAverageBalanceaverage = averageBalance(theMidClass)
    while(True):
        Sleep(1000)
        err = theAverageBalanceaverage.makeNeedAccountInfo()
        if err:
            Log('err',err)
        theAverageBalanceaverage.run('price',0.01)